# 🔍 SERP API - Google Search Results

Test the SERP (Search Engine Results Page) API:
- Google Search
- Location-specific results
- Batch queries
- Device type comparison

---

## Setup

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

API_TOKEN = os.getenv("BRIGHTDATA_API_TOKEN")
if not API_TOKEN:
    raise ValueError("Set BRIGHTDATA_API_TOKEN in .env file")

print(f"API Token: {API_TOKEN[:10]}...{API_TOKEN[-4:]}")
print("Setup complete!")

API Token: 7011787d-2...3336
Setup complete!


## Initialize Client

In [3]:
from brightdata import BrightDataClient

# Initialize client
client = BrightDataClient(token=API_TOKEN)

print("Client initialized")
print(f"Default SERP zone: {client.serp_zone}")

Client initialized
Default SERP zone: sdk_serp


---
## Test 1: Google Search

Basic Google search with default settings.

In [4]:
QUERY = "python programming tutorial"

print(f"Google Search: '{QUERY}'")
print("Location: United States\n")

async with client:
    result = await client.search.google(
        query=QUERY,
        location="United States",
        num_results=10
    )

print(f"Success: {result.success}")
print(f"Search Engine: {result.search_engine}")
print(f"Total Found: {result.total_found}")

if result.success and result.data:
    print("\n--- Top 5 Results ---")
    for i, item in enumerate(result.data[:5]):
        print(f"\n{i+1}. {item.get('title', 'N/A')}")
        print(f"   URL: {item.get('url', 'N/A')[:60]}...")
        desc = item.get('description', 'N/A')
        print(f"   Description: {desc[:80]}..." if len(str(desc)) > 80 else f"   Description: {desc}")
else:
    print(f"\nError: {result.error}")

Google Search: 'python programming tutorial'
Location: United States

Success: True
Search Engine: google
Total Found: None

--- Top 5 Results ---

1. The Python Tutorial — Python 3.14.2 documentation
   URL: https://docs.python.org/3/tutorial/index.html...
   Description: This tutorial introduces the reader informally to the basic concepts and feature...

2. Python Tutorial
   URL: https://www.w3schools.com/python/...
   Description: Learn Python. Python is a popular programming language. Python can be used on a ...

3. Python For Beginners
   URL: https://www.python.org/about/gettingstarted/...
   Description: Welcome! Are you completely new to programming? If not then we presume you will ...

4. Python Tutorial
   URL: https://www.tutorialspoint.com/python/index.htm...
   Description: This Python tutorial gives a complete understanding of Python programming langua...

5. Best Python tutorial for beginners in 2024? : r/learnpython
   URL: https://www.reddit.com/r/learnpython/comments

---
## Test 2: Location-Specific Search

Get location-specific search results.

In [7]:
QUERY = "best restaurants near me"

locations = ["New York, United States", "London, United Kingdom", "Tokyo, Japan"]

print(f"Query: '{QUERY}'")
print("Testing different locations...\n")

async with client:
    for location in locations:
        print(f"=== Location: {location} ===")
        result = await client.search.google(
            query=QUERY,
            location=location,
            num_results=3
        )
        
        if result.success and result.data:
            for i, item in enumerate(result.data[:3]):
                title = item.get('title', 'N/A')[:60]
                url = item.get('url', 'N/A')
                print(f"  {i+1}. {title}")
                print(f"     {url}")
        else:
            print(f"  Error: {result.error}")
        print()

Query: 'best restaurants near me'
Testing different locations...

=== Location: New York, United States ===
  1. Best Restaurants Near Me
     https://www.tripadvisor.com/Restaurants
  2. Book the best restaurants nearby
     https://www.opentable.com/nearby
  3. TOP 10 BEST Restaurants in San Francisco, CA
     https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA

=== Location: London, United Kingdom ===
  1. Best Restaurants Near Me
     https://www.tripadvisor.com/Restaurants
  2. Book the best restaurants nearby
     https://www.opentable.com/nearby
  3. 12 AMAZING Restaurants In Cherry Creek For Food Lovers ...
     https://nomadicfoodist.com/best-restaurants-in-cherry-creek/

=== Location: Tokyo, Japan ===
  1. Best Restaurants Near Me
     https://www.tripadvisor.com/Restaurants
  2. Best Restaurants Near Me - January 2026
     https://www.yelp.com/nearme/restaurants
  3. 12 AMAZING Restaurants In Cherry Creek For Food Lovers ...
     https://nomadicfoo

---
## Test 3: Batch Queries

Search multiple queries at once.

In [9]:
QUERIES = [
    "python web scraping",
    "javascript async await",
    "data science tools"
]

print(f"Batch Google Search: {len(QUERIES)} queries\n")

async with client:
    results = await client.search.google(
        query=QUERIES,
        location="United States",
        num_results=5
    )

print(f"Results: {len(results)} responses\n")

for i, result in enumerate(results):
    query = QUERIES[i]
    print(f"=== Query: '{query}' ===")
    print(f"Success: {result.success}")
    
    if result.success and result.data:
        print(f"Results found: {len(result.data)}")
        for j, item in enumerate(result.data[:2]):
            title = item.get('title', 'N/A')[:60]
            url = item.get('url', 'N/A')
            print(f"  {j+1}. {title}")
            print(f"     {url}")
    else:
        print(f"Error: {result.error}")
    print()

Batch Google Search: 3 queries

Results: 3 responses

=== Query: 'python web scraping' ===
Success: True
Results found: 8
  1. Python Web Scraping Tutorial
     https://www.geeksforgeeks.org/python/python-web-scraping-tutorial/
  2. How to start Web scraping with python? : r/learnpython
     https://www.reddit.com/r/learnpython/comments/qzr8ir/how_to_start_web_scraping_with_python/

=== Query: 'javascript async await' ===
Success: True
Results found: 8
  1. async function - JavaScript - MDN Web Docs
     https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/async_function
  2. Async/await
     https://javascript.info/async-await

=== Query: 'data science tools' ===
Success: True
Results found: 9
  1. 13 Essential Data Science Tools (And How to Use Them)
     https://learning.linkedin.com/resources/learning-tech/how-to-use-13-essential-data-science-tools
  2. If you had to list a “tier list” of software that data scien
     https://www.reddit.com/r/datascience/com

---
## Test 4: Device Type (Desktop vs Mobile)

Compare desktop and mobile search results.

In [10]:
QUERY = "weather today"

print(f"Query: '{QUERY}'")
print("Comparing desktop vs mobile results...\n")

async with client:
    # Desktop search
    print("=== Desktop ===")
    desktop_result = await client.search.google(
        query=QUERY,
        device="desktop",
        num_results=5
    )
    
    if desktop_result.success and desktop_result.data:
        for i, item in enumerate(desktop_result.data[:3]):
            print(f"  {i+1}. {item.get('title', 'N/A')[:50]}")
    else:
        print(f"  Error: {desktop_result.error}")
    
    print("\n=== Mobile ===")
    mobile_result = await client.search.google(
        query=QUERY,
        device="mobile",
        num_results=5
    )
    
    if mobile_result.success and mobile_result.data:
        for i, item in enumerate(mobile_result.data[:3]):
            print(f"  {i+1}. {item.get('title', 'N/A')[:50]}")

Query: 'weather today'
Comparing desktop vs mobile results...

=== Desktop ===
  1. Weather Forecast and Conditions for Austin, Texas
  2. National and Local Weather Radar, Daily Forecast, 
  3. Weather Forecast and Conditions for New York City,

=== Mobile ===
  1. Tunis, Tunisia Hourly Weather Forecast
  2. Tunis, Tunis, Tunisia Weather Forecast
  3. Tunis - BBC Weather


---
## Test 5: Timing Metadata

In [11]:
# Check timing metadata from last result
print("=== Result Metadata ===")
print(f"success: {mobile_result.success}")
print(f"search_engine: {mobile_result.search_engine}")
print(f"query: {mobile_result.query}")
print(f"country: {mobile_result.country}")
print(f"results_per_page: {mobile_result.results_per_page}")
print(f"total_found: {mobile_result.total_found}")
print("\n=== Timing ===")
print(f"trigger_sent_at: {mobile_result.trigger_sent_at}")
print(f"data_fetched_at: {mobile_result.data_fetched_at}")

if mobile_result.trigger_sent_at and mobile_result.data_fetched_at:
    duration = (mobile_result.data_fetched_at - mobile_result.trigger_sent_at).total_seconds()
    print(f"\nTotal time: {duration:.2f} seconds")

=== Result Metadata ===
success: True
search_engine: google
query: {'q': 'weather today', 'location': None, 'language': 'en'}
country: None
results_per_page: 5
total_found: None

=== Timing ===
trigger_sent_at: 2026-01-29 13:10:11.144061+00:00
data_fetched_at: 2026-01-29 13:10:13.947057+00:00

Total time: 2.80 seconds


---
## Test 6: Export Results to JSON

In [12]:
import json
from pathlib import Path

# Use the last successful result
if mobile_result.success and mobile_result.data:
    output_file = Path.cwd() / "serp_results.json"
    
    export_data = {
        "success": mobile_result.success,
        "search_engine": mobile_result.search_engine,
        "query": mobile_result.query,
        "total_found": mobile_result.total_found,
        "results_count": len(mobile_result.data),
        "results": mobile_result.data,
    }
    
    with open(output_file, "w") as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"Exported to: {output_file}")
    print(f"Results count: {len(mobile_result.data)}")
else:
    print("No successful results to export")

Exported to: /Users/ns/Desktop/projects/sdk-python/notebooks/serp_results.json
Results count: 10


---
## Summary

### SERP Methods

| Method | Description |
|--------|-------------|
| `client.search.google(query, ...)` | Google Search |

### Parameters

| Parameter | Description | Default |
|-----------|-------------|--------|
| `query` | Search query or list of queries | Required |
| `location` | Geographic location | `None` |
| `language` | Language code (en, es, ru, etc.) | `"en"` |
| `device` | `"desktop"`, `"mobile"`, `"tablet"` | `"desktop"` |
| `num_results` | Number of results | `10` |
| `zone` | SERP zone | `sdk_serp` |

### Response Fields

| Field | Description |
|-------|-------------|
| `success` | Boolean indicating success |
| `data` | List of search result items |
| `search_engine` | Which engine was used |
| `query` | The search query info |
| `total_found` | Estimated total results |
| `country` | Location used |

### Result Item Fields

| Field | Description |
|-------|-------------|
| `title` | Page title |
| `url` | Page URL |
| `description` | Snippet/description |
| `position` | Ranking position |